<a href="https://colab.research.google.com/github/Armando5347/polaridad-opinion/blob/main/polaridad_de_opinion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Imports a utilizar**

In [2]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from sklearn.neural_network import MLPClassifier
import re
import pickle

**Obtener el dataset**

In [4]:

data = pd.read_excel("Rest_Mex_2022.xlsx")
print (data)

train_split, test_split = train_test_split(
        data,
        test_size=0.2,  # 20% para prueba y 80% para entrenamiento
        random_state=0,  # Semilla para asegurar reproducibilidad
        stratify=data['Polarity']  # Mantener proporciones de clase
    )

X_train = train_split["Opinion"]
y_train = train_split["Polarity"]
X_test = test_split["Opinion"]
y_test = test_split["Polarity"]

                                       Title  ...  Attraction
0                               Pésimo lugar  ...       Hotel
1                  No vayas a lugar de Eddie  ...  Restaurant
2               Mala relación calidad-precio  ...       Hotel
3           Minusválido? ¡No te alojes aquí!  ...       Hotel
4      Es una porqueria no pierdan su tiempo  ...       Hotel
...                                      ...  ...         ...
30207          Verdadera joya arquitectónica  ...  Attractive
30208                              Romántico  ...  Attractive
30209                     Parece un castillo  ...  Attractive
30210                             Imperdible  ...  Attractive
30211                       Muy bonita vista  ...  Attractive

[30212 rows x 4 columns]


**Crear pipeline, junto con los grid_search_view para los clasificadores**

In [ ]:
clasificadores = [SVC(random_state=0), MLPClassifier(random_state=0)]
param_grid_svc = {
                'classifier__C': [0.1, 1, 10],  # Hiperparámetro C para SVM
                'classifier__kernel': ['linear', 'rbf', 'poly'],  # Tipo de kernel
                'classifier__gamma': ['scale', 'auto']  # Parámetro gamma
            }

param_grid_mlp = {
                'classifier__C': [0.1, 1, 10],  # Hiperparámetro C para SVM
                'classifier__kernel': ['linear', 'rbf', 'poly'],  # Tipo de kernel
                'classifier__gamma': ['scale', 'auto']  # Parámetro gamma
            }
for clasificador in clasificadores:
  pipe = Pipeline([('text_representation', TfidfVectorizer(token_pattern= r'(?u)\w+|\w+\n|\.|\¿|\?', ngram_range=(1,1))), ('classifier',clasificador)])
  if isinstance(clasificador, SVC):
    grid_search = GridSearchCV(pipe, param_grid_svc, cv=5, scoring='accuracy')
  else:
    grid_search = GridSearchCV(pipe, param_grid_mlp, cv=5, scoring='accuracy')
  # Entrenar el modelo con GridSearchCV
  grid_search.fit(X_train, y_train)
  print(str(grid_search.best_params_))
  y_pred = grid_search.predict(X_test)
  print(classification_report(y_test, y_pred))
